In [ ]:
%pylab inline

In [ ]:
import scipy as sp
import scipy.io as scio
import tables
import pandas as pd
import os
import numpy as np
import mne

import sys
from sklearn.preprocessing import scale
import h5io

In [76]:
import loadmat
import os
import mne
import numpy as np

In [5]:
SJdir = '/home/knight/matar/MATLAB/DATA/Avgusta'

In [11]:
filename = os.path.join(SJdir, 'Subjs', 'ST27', 'DecisionVis', 'gdat_notch.mat')
data_dict = loadmat.loadmat(filename)
gdat = data_dict['gdat']

In [31]:
filename = os.path.join(SJdir, 'Subjs', 'ST27', 'DecisionVis', 'subj_globals.mat')
data_dict = loadmat.loadmat(filename)
elecs = data_dict['elecs']
srate = data_dict['srate']

In [32]:
filename = os.path.join(SJdir, 'Subjs', 'ST27', 'DecisionVis', 'onsets_stim.mat')
data_dict = loadmat.loadmat(filename)
onsets_stim = data_dict['onsets_stim']

filename = os.path.join(SJdir, 'Subjs', 'ST27', 'DecisionVis', 'onsets_resp.mat')
data_dict = loadmat.loadmat(filename)
onsets_resp = data_dict['onsets_resp']



In [57]:
info = mne.create_info([str(e) for e in (elecs-1)], srate)
data = mne.io.RawArray(gdat[elecs-1,:], info)

Creating RawArray with float64 data, n_channels=81, n_times=1672000
    Range : 0 ... 1671999 =      0.000 ...  1671.999 secs
Ready.


In [64]:
filename = os.path.join(SJdir, 'Subjs','ST27','DecisionVis', 'gdat_notch.fif')
data.save(filename, overwrite=True)

Writing /home/knight/matar/MATLAB/DATA/Avgusta/Subjs/ST27/DecisionVis/gdat_notch.fif
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Writing ...
Wri

/home/knight/matar/.local/lib/python2.7/site-packages/mne/utils.py:258: UserWarning: This filename (/home/knight/matar/MATLAB/DATA/Avgusta/Subjs/ST27/DecisionVis/gdat_notch.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
  '%s' % (fname, filetype, print_endings))


In [90]:
# Load data
ecog = mne.io.Raw(filename, preload=True, add_eeg_ref=False)
ecog = mne.io.RawArray(ecog._data, ecog.info)

Opening raw data file /home/knight/matar/MATLAB/DATA/Avgusta/Subjs/ST27/DecisionVis/gdat_notch.fif...
Isotrak not found
    Range : 0 ... 1671999 =      0.000 ...  1671.999 secs
Ready.
Reading 0 ... 1671999  =      0.000 ...  1671.999 secs...
[done]
Creating RawArray with float64 data, n_channels=81, n_times=1672000
    Range : 0 ... 1671999 =      0.000 ...  1671.999 secs
Ready.


<Raw  |  n_channels x n_times : 81 x 1672000>

In [70]:
band_type = 'one'

In [71]:
if band_type == 'multi':
    fmin, fmax = (75, 140)
    win_base = 5
    freqs = np.logspace(np.log10(fmin), np.log10(fmax), 14)
    fwins = win_base * (2 ** (freqs / fmin))
elif band_type == 'one':
    fmin, fmax = (70, 150)
    frange = fmax - fmin
    freqs = [fmin + frange/2]
    freq = freqs[0]
    fwin = frange
n_jobs = 1

In [77]:
# Make length a pow of 2
orig_times = ecog.n_times
new_len = int(2**np.ceil(np.log2(orig_times)))

# Filter for HG and extract amplitude
ecog_bands = np.zeros([len(freqs), len(ecog.ch_names), ecog.n_times])
if len(freqs) > 1:
    for i, (freq, fwin) in enumerate(zip(freqs, fwins)):
        # Extract a range of frequency bands for averaging later
        ecog_band = ecog.copy()
        ecog_band.filter(freq-fwin/2, freq+fwin/2, n_jobs=n_jobs)
        ecog_band.apply_hilbert(range(len(ecog_band.ch_names)), envelope=True,
                                n_jobs=n_jobs, n_fft=new_len)
        
        if normalize is True:
            # Scale frequency band so that the ratios of all are the same
            ecog_band_mn = ecog_band._data.mean(1)[:, None]
            ecog_band._data /= ecog_band_mn
        ecog_bands[i] = ecog_band._data.copy()

    # Now overwrite the data and save
    ecog._data = ecog_bands.mean(0)
else:
    # Don't need to worry about frequency scaling etc.
    ecog.filter(freq-fwin/2, freq+fwin/2, n_jobs=n_jobs)
    ecog.apply_hilbert(range(len(ecog.ch_names)), envelope=True,
                            n_jobs=n_jobs, n_fft=new_len)

RuntimeError: Could not find any valid channels for your Raw object. Please contact the MNE-Python developers.

In [62]:
filename = os.path.join(SJdir, 'Subjs','ST27','DecisionVis', 'data.h5')
dset = h5py.File(filename, 'w')
dset.create_dataset('gdat', data = gdat)

<Closed HDF5 file>